In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from pathlib import Path
import sys
import altair as alt

In [34]:
item_features = pd.read_csv('./data/item_features.csv')
train_sessions = pd.read_csv('./data/train_sessions.csv')
train_purchases = pd.read_csv('./data/train_purchases.csv')
train_labls = pd.read_csv('./data/train_purchases.csv')

item_features.sample(10)

,item_id,feature_category_id,feature_value_id
269400,16054,31,768
223504,13316,30,847
176570,10504,16,38
413836,24646,63,816
282691,16840,68,479
31944,1930,61,706
383500,22850,69,116
238371,14205,61,706
388608,23158,46,825
376954,22442,3,793


In [4]:
pivoted = pd.pivot_table(item_features, index='item_id', columns='feature_category_id', values='feature_value_id')
pivoted

feature_category_id,1,2,3,4,5,6,7,8,9,10,...,64,65,66,67,68,69,70,71,72,73
item_id,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,394.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,351.0,885.0,NaN,NaN,75.0,NaN
3,NaN,NaN,889.0,618.0,605.0,NaN,452.0,NaN,NaN,NaN,...,NaN,521.0,NaN,NaN,14.0,592.0,NaN,NaN,75.0,544.0
4,NaN,NaN,793.0,618.0,605.0,NaN,837.0,NaN,NaN,NaN,...,NaN,521.0,NaN,NaN,373.0,538.0,NaN,NaN,75.0,544.0
7,NaN,NaN,NaN,NaN,NaN,NaN,536.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,739.0,592.0,NaN,NaN,75.0,NaN
8,NaN,NaN,793.0,618.0,605.0,NaN,798.0,NaN,NaN,NaN,...,NaN,521.0,NaN,NaN,351.0,592.0,NaN,NaN,75.0,544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28139,NaN,NaN,793.0,618.0,605.0,NaN,798.0,NaN,NaN,NaN,...,NaN,521.0,NaN,NaN,106.0,805.0,NaN,NaN,75.0,544.0
28140,NaN,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80.0,NaN,NaN,349.0,351.0,NaN,NaN,226.0,NaN,544.0
28141,461.0,NaN,889.0,719.0,605.0,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,379.0,499.0,NaN,NaN,75.0,544.0


In [35]:
dates = pd.to_datetime(train_sessions['date'])
train_sessions['date'] = (dates - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
dates = pd.to_datetime(train_labls['date'])
train_labls['date'] = (dates - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
train = pd.concat([train_sessions, train_labls], axis=0)
train = train.sort_values(by='date')
train

,session_id,item_id,date
4005618,3747794,11234,1577836801
3699175,3458777,3868,1577836821
4005616,3747794,9214,1577836822
4649622,4350716,10861,1577836848
979909,4350716,12737,1577836931
...,...,...,...
422305,1876524,16157,1622504564
3927983,3674124,23412,1622504582
3927985,3674124,17618,1622504613
827526,3674124,15332,1622504623


In [36]:
import random
cats = ['shirt','pants','jackets','shoes','dress','T-shirt']
train['cats'] = train['session_id'].apply(lambda x: random.choice(cats))
train['label'] = 1
train = train[['label','session_id','item_id','date','cats']]
train.to_csv('./data/our/processed_train.csv', sep='\t', index=False, header=None)
train

,label,session_id,item_id,date,cats
4005618,1,3747794,11234,1577836801,T-shirt
3699175,1,3458777,3868,1577836821,shirt
4005616,1,3747794,9214,1577836822,shirt
4649622,1,4350716,10861,1577836848,T-shirt
979909,1,4350716,12737,1577836931,jackets
...,...,...,...,...,...
422305,1,1876524,16157,1622504564,T-shirt
3927983,1,3674124,23412,1622504582,T-shirt
3927985,1,3674124,17618,1622504613,shirt
827526,1,3674124,15332,1622504623,shirt


# DATA PREPERATION

In [3]:
import sys
import os
import logging
import papermill as pm
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import tensorflow.compat.v1 as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.models.deeprec.deeprec_utils import (
    prepare_hparams
)
from recommenders.datasets.amazon_reviews import download_and_extract, data_preprocessing
from recommenders.datasets.download_utils import maybe_download


from recommenders.models.deeprec.models.sequential.sli_rec import SLI_RECModel as SeqModel
####  to use the other model, use one of the following lines:
# from recommenders.models.deeprec.models.sequential.asvd import A2SVDModel as SeqModel
# from recommenders.models.deeprec.models.sequential.caser import CaserModel as SeqModel
# from recommenders.models.deeprec.models.sequential.gru4rec import GRU4RecModel as SeqModel
# from recommenders.models.deeprec.models.sequential.sum import SUMModel as SeqModel

#from recommenders.models.deeprec.models.sequential.nextitnet import NextItNetModel

from recommenders.models.deeprec.io.sequential_iterator import SequentialIterator
#from recommenders.models.deeprec.io.nextitnet_iterator import NextItNetIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.9.10 (tags/v3.9.10:f2f3f53, Jan 17 2022, 15:14:21) [MSC v.1929 64 bit (AMD64)]
Tensorflow version: 2.7.3


In [4]:
yaml_file = './rec/sli_rec.yaml' 

In [5]:
EPOCHS = 10
BATCH_SIZE = 400
RANDOM_SEED = SEED  # Set None for non-deterministic result
train_num_ngs = 4 # number of negative instances with a positive instance for training
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 9 # number of negative instances with a positive instance for testing
sample_rate = 0.01 # sample a small item set for training and testing here for fast example

'.\\rec\\datasets'

In [20]:
import rec.preprocessing as ff
from importlib import reload


def data_preprocessing(
    reviews_file,
    meta_file,
    train_file,
    valid_file,
    test_file,
    user_vocab,
    item_vocab,
    cate_vocab,
    sample_rate=0.01,
    valid_num_ngs=4,
    test_num_ngs=9,
    is_history_expanding=True,
):
    """Create data for training, validation and testing from original dataset

    Args:
        reviews_file (str): Reviews dataset downloaded from former operations.
        meta_file (str): Meta dataset downloaded from former operations.
    """
    reviews_output = ff._reviews_preprocessing(reviews_file)
    meta_output = ff._meta_preprocessing(meta_file)
    instance_output = ff._create_instance(reviews_output, meta_output)
    ff._create_item2cate(instance_output)
    # sampled_instance_file = ff._get_sampled_data(instance_output, sample_rate=sample_rate)
    preprocessed_output = ff._data_processing(instance_output)
    if is_history_expanding:
        ff._data_generating(preprocessed_output, train_file, valid_file, test_file)
    else:
        ff._data_generating_no_history_expanding(
            preprocessed_output, train_file, valid_file, test_file
        )
    ff._create_vocab(train_file, user_vocab, item_vocab, cate_vocab)
    ff._negative_sampling_offline(
        instance_output, valid_file, test_file, valid_num_ngs, test_num_ngs
    )
    
# data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)

In [53]:
our_path = os.path.join('.','data','our')
train_file = os.path.join(our_path,r'train_data')
valid_file = os.path.join(our_path,r'valid_data')
test_file = os.path.join(our_path,r'test_data')
user_vocab = os.path.join(our_path,r'user_vocab.pkl')
item_vocab = os.path.join(our_path,r'item_vocab.pkl')
cate_vocab = os.path.join(our_path,r'category_vocab.pkl')
output_file = os.path.join(our_path,r'output.txt')
ff = reload(ff)
a = ff._data_processing(os.path.join(our_path,'processed_train.csv'))
ff._data_generating(a, train_file, valid_file, test_file)
ff._create_vocab(train_file, user_vocab, item_vocab, cate_vocab)
ff._create_item2cate(os.path.join(our_path,'processed_train.csv'))
ff._negative_sampling_offline(
        os.path.join(our_path,'processed_train.csv'), valid_file, test_file, valid_num_ngs, test_num_ngs
    )

start data processing...
creating item2cate dict
start valid negative sampling
start test negative sampling


In [54]:
hparams = prepare_hparams(yaml_file, 
                          embed_l2=0., 
                          layer_l2=0., 
                          learning_rate=0.001,  # set to 0.01 if batch normalization is disable
                          epochs=EPOCHS,
                          batch_size=BATCH_SIZE,
                          show_step=20,
                          MODEL_DIR=os.path.join(our_path, "model/"),
                          SUMMARIES_DIR=os.path.join(our_path, "summary/"),
                          user_vocab=user_vocab,
                          item_vocab=item_vocab,
                          cate_vocab=cate_vocab,
                          need_sample=True,
                          train_num_ngs=train_num_ngs, # provides the number of negative instances for each positive instance for loss computation.
            )
            
input_creator = SequentialIterator

In [55]:
model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)

c:\Users\rietesh.amminabhavi\AppData\Local\Programs\Python\Python39\lib\site-packages\recommenders\models\deeprec\models\base_model.py:701: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  curr_hidden_nn_layer = tf.compat.v1.layers.batch_normalization(
c:\Users\rietesh.amminabhavi\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\legacy_tf_layers\normalization.py:455: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs, training=training)


In [56]:
print(model.run_eval(test_file, num_ngs=test_num_ngs)) 

{'auc': 0.5094, 'logloss': 0.6931, 'mean_mrr': 0.2817, 'ndcg@2': 0.1524, 'ndcg@4': 0.2404, 'ndcg@6': 0.3127, 'group_auc': 0.5081}
